In [5]:
import pandas as pd
from transformers import pipeline

In [ ]:
classifier = pipeline('zero-shot-classification', model='facebook/bart-large-mnli')
labels = ["bussiness","technology","science","sports","media and entertainment","politics","health","crime","accident","environment","art","literature","tragedy","education","fashion","food","travel","military","real estates","history","religion","celebrity"]
hypothesis_template = "this text is about {}"


c:\Users\alkrd\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\alkrd\.cache\huggingface\hub\models--facebook--bart-large-mnli. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class

In [58]:
# data = pd.read_csv('../preprocessed_data/cleaned_data_USA.csv')
data = pd.read_csv('./test.csv')
dataFrame = data[['date','traffic','newsSnippet']]
strings_list = [item[2] for item in dataFrame.values.tolist()]
loop = range(len(strings_list))



In [59]:
results = []
batch_size = 4
for i in range(0, len(strings_list), batch_size):
    batch = strings_list[i:i+batch_size]
    predictions = classifier(batch, labels, hypothesis_template=hypothesis_template, multi_class=True)
    for j, prediction in enumerate(predictions):
        top_label = prediction['labels'][0]
        top_score = prediction['scores'][0]
        row_data = (dataFrame.iloc[i+j]['date'], dataFrame.iloc[i+j]['traffic'], dataFrame.iloc[i+j]['newsSnippet'], top_label, top_score)
        results.append(row_data)

df_results = pd.DataFrame(results, columns=['date', 'traffic', 'newsTitle', 'predicted_label', 'score'])
df_results.to_csv('./classification_output.csv', index=False)

The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.


In [ ]:
# sequence = "Ohio State football players safe accounted for after oncampus attack,COLUMBUS Ohio  An attack on Ohio States campus left 11 people injured Monday morning They were stabbed hit by a car or otherwise injured before the suspect was shot dead school and hospital officials said All Ohio State football players are"
# prediction = classifier(sequence, labels, hypothesis_template=hypothesis_template, multi_class=True)
# prediction['labels'][0]

The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.


'crime'